# Agentic RAG: turbocharge RAG with query reformulation and self-query

- **Retrieval-Augmented-Generation (RAG)** is `using an LLM to answer a user query, but basing the answer on information retrieved from a knowledge base`.
- It has many advantages over using a vanilla or fine-tuned LLM: to name a few, it allows to ground the answer on true facts and reduce confabulations, it allows to provide the LLM with domain-specific knowledge, and it allows fine-grained control of access to information from the knowledge base.

## Install required dependencies

In [1]:
!pip install pip pandas langchain langchain-community sentence-transformers faiss-cpu smolagents --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load the dataset

In [3]:
!pip install -U datasets

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [datasets]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

## Processing the dataset and storing it into a vector database

In [5]:
from tqdm import tqdm
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]}) for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)

print("Embedding documents... This should take a few minutes (5 minutes on MacBook with M1 Pro/ 10 min on Windows)")
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Splitting documents...


...

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Build the agentic RAG system

### Define the RetrieverTool to retrieve information from the knowledge base

In [6]:
from smolagents import Tool
from langchain_core.vectorstores import VectorStore


class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

### Create our agent

The agent will need these arguments upon initialization:

- `tools`: a list of tools that the agent will be able to call.
- `model`: the LLM that powers the agent.

In [7]:
from smolagents import InferenceClientModel, ToolCallingAgent

model = InferenceClientModel("meta-llama/Llama-3.1-8B-Instruct")
# model = InferenceClientModel("Qwen/Qwen2.5-72B-Instruct")

retriever_tool = RetrieverTool(vectordb)
agent = ToolCallingAgent(tools=[retriever_tool], model=model)

In [8]:
agent_output = agent.run("How can I push a model to the Hub?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How can I push a model to the Hub?                                                                              │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-3.1-8B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'How to push a model to the Hub'}                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved documents:
===== Document 0 =====
.push_to_hub(). So this is more of a once-off method - it's not called regularly during training. You can just call
this manually whenever you want to upload a model to the hub. So we recommend running this after the end of 
training, just to make sure that you have a commit message just to guarantee that this was the final version of the
model at the end of training. And it just makes sure that you're working with the definitive end-of-training model 
and not accidentally using a model that's from a checkpoint somewhere along the way===== Document 1 =====
```py
>>> trainer.push_to_hub()
```
</pt>
<tf>
Share a model to the Hub with |`PushToHubCallback`]. In the |`PushToHubCallback`] function, add:

- An output directory for your model.
- A tokenizer.
- The `hub_model_id`, which is your Hub username and model name.

```py
>>> from transformers import PushToHubCallback

>>> push_to_hub_callback = PushToHubCallback(
...     output_dir="./your_model_save_path", tokenizer=tokenizer, hub_model_id="your-username/my-awesome-model"
... )
```===== Document 2 =====
Finally, if you want, you can push your model up to the hub. Here, we'll push it up if you specified 
`push_to_hub=True` in the training configuration. Note that in order to push to hub, you'll have to have git-lfs 
installed and be logged into your Hugging Face account (which can be done via `huggingface-cli login`).

```python
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "dataset": 'beans',
    "tags": |'image-classification'],
}

if training_args.push_to_hub:
    trainer.push_to_hub('🍻 cheers', **kwargs)
else:
    trainer.create_model_card(**kwargs)
```===== Document 3 =====
--push_to_hub
```===== Document 4 =====
The `push_to_hub=True` argument will allow us to push the model to the Hub after training; you'll find the 
repository under your user profile in the location defined by `output_dir`. Note that you can specify the name of 
the repository you want to push to with the `hub_model_id` argument (in particular, you will have to use this 
argument to push to an organization). For instance, when we pushed the model to the |`huggingface-course` 
organization](https://huggingface.co/huggingface-course), we added 
`hub_model_id="huggingface-course/mt5-finetuned-amazon-en-es"` to `Seq2SeqTrainingArguments`.===== Document 5 =====
{/if}

At a lower level, accessing the Model Hub can be done directly on models, tokenizers, and configuration objects via
their `push_to_hub()` method. This method takes care of both the repository creation and pushing the model and 
tokenizer files directly to the repository. No manual handling is required, unlike with the API we'll see below.

To get an idea of how it works, let's first initialize a model and a tokenizer:

{#if fw === 'pt'}
```py
from transformers import AutoModelForMaskedLM, AutoTokenizer

checkpoint = "camembert-base"===== Document 6 =====
In addition, we set up a `PushToHubCallback` that will save the model to the Hub after each epoch. You can specify 
the name of the repository you want to push to with the `hub_model_id` argument (in particular, you will have to 
use this argument to push to an organization). For instance, to push the model to the |`huggingface-course` 
organization](https://huggingface.co/huggingface-course), we added 
`hub_model_id="huggingface-course/distilbert-finetuned-imdb"`. By default, the repository used will be in your 
namespace and named after the output directory you set, so in our case it will be 
`"lewtun/distilbert-finetuned-imdb"`.

[Step 1: Duration 3.84 seconds| Input tokens: 1,197 | Output tokens: 29]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "```py>>> from transformers import PushToHubCallback,   │
│ Trainer\\'\\\\"}                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: ```py>>> from transformers import PushToHubCallback, Trainer\'\\

[Step 2: Duration 2.68 seconds| Input tokens: 3,325 | Output tokens: 65]

Final output:
```py>>> from transformers import PushToHubCallback, Trainer\'\\


## Evaluating Agentic RAG vs. standard RAG


In [9]:
import logging

# set the root logger
logging.getLogger().setLevel(logging.WARNING)


In [10]:
import logging

eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

In [11]:
outputs_agentic_rag = []

for example in tqdm(eval_dataset):
    question = example["question"]

    enhanced_question = f"""Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".

Question:
{question}"""
    answer = agent.run(enhanced_question)
    print("=======================================================")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f'True answer: {example["answer"]}')

    results_agentic = {
        "question": question,
        "true_answer": example["answer"],
        "source_doc": example["source_doc"],
        "generated_answer": answer,
    }
    outputs_agentic_rag.append(results_agentic)

  0%|          | 0/65 [00:00<?, ?it/s]

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Using the information contained in your knowledge base, which you can access with the 'retriever' tool,         │
│ give a comprehensive answer to the question below.                                                              │
│ Respond only to the question asked, response should be concise and relevant to the question.                    │
│ If you cannot find information, do not give up and try calling your retriever again with different arguments!   │
│ Make sure to have covered the question completely by calling the retriever tool several times with semantically │
│ different queries.                                                                                              │
│ Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model    │
│ from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".                               │
│                                                                                                                 │
│ Question:                                                                                                       │
│ What architecture is the `tokenizers-linux-x64-musl` binary designed for?                                       │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-3.1-8B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'tokenizers-linux-x64-musl binary architecture'}            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved documents:
===== Document 0 =====
`tokenizers-linux-x64-musl`

This is the **x86_64-unknown-linux-musl** binary for `tokenizers`===== Document 1 =====
`tokenizers-linux-arm64-musl`

This is the **aarch64-unknown-linux-musl** binary for `tokenizers`===== Document 2 =====
`tokenizers-linux-x64-gnu`

This is the **x86_64-unknown-linux-gnu** binary for `tokenizers`===== Document 3 =====
`tokenizers-linux-arm64-gnu`

This is the **aarch64-unknown-linux-gnu** binary for `tokenizers`===== Document 4 =====
`tokenizers-freebsd-x64`

This is the **x86_64-unknown-freebsd** binary for `tokenizers`===== Document 5 =====
`tokenizers-win32-x64-msvc`

This is the **x86_64-pc-windows-msvc** binary for `tokenizers`===== Document 6 =====
`tokenizers-android-arm64`

This is the **aarch64-linux-android** binary for `tokenizers`

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error while generating output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/meta-llama/Llama-3.1-8B-Instruct/v1/chat/completions (Request ID:
Root=1-682d5f3c-0527d6a66f4ee137445078fc;7ac9aed3-6335-46a6-b57d-aa27046d59fa)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.12 seconds| Input tokens: 2,027 | Output tokens: 24]

 12%|█▏        | 8/65 [00:52<06:12,  6.54s/it]


AgentGenerationError: Error while generating output:
402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/meta-llama/Llama-3.1-8B-Instruct/v1/chat/completions (Request ID: Root=1-682d5f3c-0527d6a66f4ee137445078fc;7ac9aed3-6335-46a6-b57d-aa27046d59fa)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.